### Seminar #3. The Materials Project API

#### Goals
- Overview of the Materials Project database
- Learn how to use the Materials Project API
- Learn how to manipulate the collected data
- Learn how to calculate phase diagrams with pymatgen
- Learn how to calculate eleectrochemical stability window


#### Resources
[MP API: Getting started](https://docs.materialsproject.org/downloading-data/using-the-api/getting-started)

#### Before the seminar let's have a look at the Materials Project's [website](https://next-gen.materialsproject.org/materials/mp-1960#thermodynamic_stability)

Install mp_api

In [ ]:
!pip install mp_api

Alternatively

In [ ]:
!git clone https://github.com/materialsproject/api
!cd api
!pip install -e .

## Part 1. Intro + Formation energy

In [3]:
# immports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mp_api.client import MPRester

We will use the construction below for query the data. Using Python's ```with``` is recommended for session management. You need your api_key. Visit this [page](https://next-gen.materialsproject.org/api#api-key) to get it.

In [4]:
api_key = 'your_api_key'
api_key = 'HbSHn48X1CSvBzRa4DOX5E9rhb5Tcc03'
with MPRester(api_key) as mpr:
    # do stuff
    pass


Problem loading MPContribs client: __init__() got an unexpected keyword argument 'session'


Query data by the mp_id

In [ ]:
with MPRester(api_key) as mpr:
    docs = mpr.materials.summary.search(
        material_ids=["mp-19017", "mp-13", "mp-22526"]
    )

Three entries were collected

In [ ]:
len(docs)

Let's take a look at the fields that have been collected

In [ ]:
docs[0].__dict__.keys()

In [ ]:
doc = docs[2]
doc.nsites == doc.__dict__['nsites']

In [ ]:
st = doc.structure

In [ ]:
st

You can save the structures and download them as follows

In [ ]:
import os
os.makedirs('cifs', exist_ok = True)
for doc in docs:
    st = doc.structure
    mp_id = doc.material_id
    st.to(f'cifs/{mp_id}.cif')

In [ ]:
!zip -r cifs.zip cifs

In [ ]:
from google.colab import files
files.download('cifs.zip')

The Materials Project uses corrections to recalculate the total energy. It is done "to better model energies across diverse chemical spaces".


Have a look at the [documentaion](https://docs.materialsproject.org/methodology/materials-methodology/thermodynamic-stability/thermodynamic-stability)

In [ ]:
doc.energy_per_atom, doc.uncorrected_energy_per_atom

We can collect a data corresponding to the materials that match our desired properties by passing the specific keywords to the ```search``` method.

In [ ]:
with MPRester(api_key) as mpr:
    docs = mpr.materials.summary.search(
                              fields = [
                                        'structure',
                                        'material_id',
                                        'nsites',
                                        'elements',
                                        'nelements',
                                        #'composition',
                                        #'composition_reduced',
                                        'formula_pretty',
                                        'chemsys',
                                        #'volume',
                                        #'density',
                                        #'density_atomic',
                                        #'symmetry',
                                        #'last_updated',
                                        'energy_above_hull',
                                        'is_stable',
                                        'band_gap',
                                        'cbm',
                                        'vbm',
                                        'efermi',
                                        # this is not all possbile fields
                                        ],
                              exclude_elements = [
                                        # there is a limit of 15 characters for this query
                                                    'U'
                                                       ],
                              volume = (0, 100),
                              is_metal = False,
                              energy_above_hull = (0, 0.01),
                              band_gap = (3.0, 100.0),
                              possible_species = ['Mg2+'],
)

Query data from the Na-P-S system and its subsystems

In [ ]:
# create list with the chemical systems of interest
import itertools
elements = ['Na', 'P', 'S']

chemsys_list = []
for i in range(1, len(elements)+1):
    els = [list(x) for x in itertools.combinations(elements, i)]
    chemsys_list.extend(els)


In [ ]:
chemsys_list = ['-'.join(chemsys) for chemsys in chemsys_list]

In [ ]:
chemsys_list

In [ ]:
from mp_api.client import MPRester

docs = []

with MPRester(api_key) as mpr:
    for chemsys in chemsys_list:
        print(chemsys)
        docs.extend(
                    mpr.materials.summary.search(
                                                chemsys= chemsys,
                                                fields=[
                                                        "material_id",
                                                        "band_gap",
                                                        "chemsys",
                                                        "energy_per_atom",
                                                        "uncorrected_energy_per_atom",
                                                        "structure",
                                                        "composition",
                                                        "formula_pretty",
                                                        "nsites",
                                                        "e_total", # this is not the total energy! it is the dielectric tensor
                                                        "formation_energy_per_atom",
                                                        "energy_above_hull",
                                                        ]
                                                ,
                                                )
                    )

In [ ]:
len(docs)

In [ ]:
docs[0]

#### Task 1: DataFrame
Convert collected docs to the pandas DataFrame

In [ ]:
table = pd.DataFrame()


In [ ]:
table[table.chemsys == 'P']

### Formation energy

Have a look at this [page](https://docs.materialsproject.org/methodology/materials-methodology/thermodynamic-stability/phase-diagrams-pds).


"Formation energy is the energy change upon reacting to form a phase of interest from its constituent components"

$ΔE_{F} = E - ∑^{N}_{i}n_{i}μ_{i}$, where
- $E$ is the total energy of the system
- $N$ is the number of components in the system
- $\mu$ is the total energy of a component $i$
- $n$ is the total number of moles of a component $i$


### Task 2: Groupby

Group your table by 'formula_pretty', keeping only the structure with minimum energy for a given composition.

Hint: It's a one-liner

In [ ]:
# table_grouped = 

### Task 3: Formation energy

Calculate the formation energy per atom for each structure in the grouped dataframe

In [ ]:
terminal_elements_energies = {
    'Na': table_grouped[table_grouped.formula_pretty == 'Na'].energy_per_atom.values[0],
    'P': table_grouped[table_grouped.formula_pretty == 'P'].energy_per_atom.values[0],
    'S': table_grouped[table_grouped.formula_pretty == 'S'].energy_per_atom.values[0],
}

terminal_elements_energies

In [ ]:


def calculate_formation_energy(structure, energy_per_atom, terminal_elements_energies):

    """
    This function calculates the formation energy of a given structure

    Params
    ------

    structure: pymatgen's Structure object
        system for which the formation energy is calculated

    energy_per_atom: float
        the calculated total energy of the system
        divided by number of atoms in the structure

    terminal_elements_energies: dictionary
        dictionary with elements and corresponding total energies, e.g. {'Li', -3.3, 'O': -9.1}


    Returns
    -------

    formation energy of the given system

    """
    return e_formation

formation_energy = []
for st, epa in zip(table_grouped.structure, table_grouped.energy_per_atom):
    formation_energy.append(calculate_formation_energy(st, epa, terminal_elements_energies))



In [ ]:
table_grouped['formation_energy_per_atom_custom'] = formation_energy
table_grouped[['formula_pretty', 'nsites', 'energy_per_atom', 'formation_energy_per_atom', 'formation_energy_per_atom_custom', 'energy_above_hull']][table_grouped.chemsys == 'Na-P-S']

As you can see, the calculated formation energies differ from that of the materials projects database. Again, this is due to the corrections used in the MP. Where can we get this corrections?

[Anion and GGA/GGA+U mixing](https://docs.materialsproject.org/methodology/materials-methodology/thermodynamic-stability/thermodynamic-stability/anion-and-gga-gga+u-mixing)

In the case of simple systems the corrected and uncorrected formation energies are the same. Keep it in mind when comparing your own data (i.e. VASP calculations) with the MP data.

In [ ]:
table_grouped['formation_energy_per_atom_custom'] = formation_energy
table_grouped[['formula_pretty', 'formation_energy_per_atom', 'formation_energy_per_atom_custom']][table_grouped.chemsys == 'Na-P']

We can get more details on corrections and the calculation scheme using ```mpr.get_entries_in_chemsys``` to query "entries"

In [ ]:
with MPRester(api_key) as mpr:
    entries = mpr.get_entries_in_chemsys(elements=["Na", "P", "S"],
                                         additional_criteria={"thermo_types": ["GGA_GGA+U_R2SCAN"]}
                                         )

In [ ]:
entries[87].energy, entries[87].correction

In [ ]:
entries[87].as_dict()

## Part 2. Phase diagrams

- We already know how to calculate the formation energy of a system. What about its stability in the presence of competing phases?

- In order to assess the thermodynamic stability of the phases present in a given chemical system under given conditions, a phase diagram is used.

- We will use Pymatgen for constructing phase diagrams



![bg](https://docs.materialsproject.org/~gitbook/image?url=https%3A%2F%2F2369879881-files.gitbook.io%2F%7E%2Ffiles%2Fv0%2Fb%2Fgitbook-x-prod.appspot.com%2Fo%2Fspaces%252F-MhdHkeirg8PPHDHWitE%252Fuploads%252FUUwwBplE1nvT2luYq6YE%252F10853_2022_6915_Fig1_HTML.webp%3Falt%3Dmedia%26token%3D3b882c43-23f3-487b-b8e1-13e35235021d&width=768&dpr=2&quality=100&sign=f9ceca5&sv=1)



![bg](https://media.geeksforgeeks.org/wp-content/uploads/20240720163903/Convex-Hull.jpg)

"The convex hull is the smallest convex set that encloses all the points, forming a convex polygon"

[Source](https://www.geeksforgeeks.org/convex-hull-algorithm/)

In [ ]:
import pandas as pd
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
from pymatgen.entries.compatibility import MaterialsProjectCompatibility

# collect the data
with MPRester(api_key) as mpr:
    unprocessed_entries = mpr.get_entries_in_chemsys(elements=["Na", "P"],
                                         additional_criteria={"thermo_types": ["GGA_GGA+U_R2SCAN"]}
                                        )

# create the phase diagram
diagram = PhaseDiagram(unprocessed_entries)

# calcualte formation energy using Pymatgen
formation_energies = []
for entry in unprocessed_entries:
    formation_energies.append(diagram.get_form_energy_per_atom(entry))

#compat = MaterialsProjectCompatibility()
#processed_entries = compat.process_entries(unprocessed_entries)  # filter and add energy corrections


In [ ]:
len(unprocessed_entries)

In [ ]:
plotter = PDPlotter(diagram, show_unstable=True)
plotter.show()

In [ ]:
stable_entries = diagram.stable_entries

diagram.get_e_above_hull(unprocessed_entries[6])

In [ ]:
import pandas as pd
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
from pymatgen.entries.compatibility import MaterialsProjectCompatibility

diagram = PhaseDiagram(entries)
formation_energies = []
for entry in entries:
    formation_energies.append(diagram.get_form_energy_per_atom(entry))


with MPRester(api_key) as mpr:
    unprocessed_entries = mpr.get_entries_in_chemsys(
                                                    elements=["Na", "S", "P"],
                                                    additional_criteria={"thermo_types": ["GGA_GGA+U"]}

                                                     )
compat = MaterialsProjectCompatibility()
#processed_entries = compat.process_entries(unprocessed_entries)  # filter and add energy corrections
diagram = PhaseDiagram(unprocessed_entries)




In [ ]:
plotter = PDPlotter(diagram, show_unstable=False)
plotter.show()

In [ ]:
import pandas as pd
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
from pymatgen.entries.compatibility import MaterialsProjectCompatibility

diagram = PhaseDiagram(entries)
formation_energies = []
for entry in entries:
    formation_energies.append(diagram.get_form_energy_per_atom(entry))


with MPRester(api_key) as mpr:
    unprocessed_entries = mpr.get_entries_in_chemsys(
                                                    elements=["Na", "Sn", "Ge", "P"],
                                                    additional_criteria={"thermo_types": ["GGA_GGA+U"]}

                                                     )
#compat = MaterialsProjectCompatibility()
#processed_entries = compat.process_entries(unprocessed_entries)  # filter and add energy corrections
diagram = PhaseDiagram(unprocessed_entries)


In [ ]:
plotter = PDPlotter(diagram, show_unstable=False)
plotter.show()

### Task 4: Local environment

- Collect stable Li, Na, K-based phosphates
- Calculate average X-O distance for the collected compounds (X = Li, Na, K)

- Calculate effective coordination number of X in the collected crystals


In [74]:
with MPRester(api_key) as mpr:
    docs = mpr.materials.summary.search(
                              fields = [
                                        'structure',
                                        'material_id',                            
                                        'formula_pretty',
                                        'chemsys',
                                        'band_gap',
                                        ],
                              #volume = (0, 200),
                              is_stable = True,
                              chemsys = ['K-P-O']
)

Retrieving SummaryDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

In [75]:
def _effective_coordination_number(areas):
    return np.square(np.array(areas).sum()) / np.square(areas).sum()
    

In [76]:
from pymatgen.analysis.local_env import VoronoiNN

CN = []
average_distance = []

for doc in docs:

    # get structure
    st = doc.structure

    # create calculator
    calc = VoronoiNN()

    # itetrate over the sites
    for i in range(len(st)):

        # if element sitting at the curent site != 'K'
        if str(st.sites[i].specie) != 'K':
            continue
            
        # else collect features related to the Voronoi polyhedron 
        # constructed around the site
        poly_data = calc.get_voronoi_polyhedra(st, i)
        
        # features we want to collect
        areas = []
        distances = []
        for nn in poly_data.keys():
            areas.append(poly_data[nn]['area']) # area of each face is added to the list
            distances.append(poly_data[nn]['face_dist'] * 2) # multiple by 2 to get bond distance
        
        CN.append(_effective_coordination_number(areas)) # append effective CN
        average_distance.append(np.mean(distances))
        


In [69]:
np.mean(CN), np.mean(average_distance) # Li

(6.125781784788985, 2.9731320626308335)

In [73]:
np.mean(CN), np.mean(average_distance) # Na

(7.6790668500140935, 3.1695589807488913)

In [77]:
np.mean(CN), np.mean(average_distance) # K

(9.948639560746765, 3.4212910916824675)

## Part 3. Electrochemical stability of a solid electrolyte

"The electrochemical window (EW) of a substance is the electrode electric potential range between which the substance is neither oxidized nor reduced." ([link](https://en.wikipedia.org/wiki/Electrochemical_window))

In [ ]:
# query entries
chemsys = 'Na-P-S'

with MPRester(api_key = api_key) as mp_rester:

    entry = mp_rester.get_entry_by_material_id('mp-28782')[0] # Na3PS4
    entries = mp_rester.get_entries_in_chemsys(chemsys, compatible_only=True)

In [ ]:
from pymatgen.core import Element
import re

# construct phase diagram
ref_element = 'Na'
phase_diagram = PhaseDiagram([entry] + entries)
Na_entries = [e for e in entries if e.composition.reduced_formula == ref_element]
uNa0 = min(Na_entries, key=lambda e: e.energy_per_atom).energy_per_atom
el_profile = phase_diagram.get_element_profile(Element(ref_element), entry.composition)

# plot profile
fig, ax = plt.subplots(dpi = 150)

tol = 1e-3
limits = []
for i, d in enumerate(el_profile):



    v = -(d["chempot"] - uNa0)
    print(i, d['reaction'], v)

    if i != 0:
        ax.plot([x2, x2], [y1, d["evolution"] / 2.0], "k", linewidth=2)
        print(x2)
    x1 = v
    y1 = d["evolution"] / 2.0
    if abs(d['evolution']) < tol:
        limits.append(v)
        limits.append(-(el_profile[i + 1]['chempot'] - uNa0))

    if i != len(el_profile) - 1:
        x2 = -(el_profile[i + 1]["chempot"] - uNa0)
    else:
        x2 = 5.0

    if i in [k for k in range(len(el_profile))]:
        products = [
            re.sub(r"(\d+)", r"$_{\1}$", p.reduced_formula)
            for p in d["reaction"].products
            if p.reduced_formula != ref_element
        ]

        shift = 0.05

        ax.annotate(
            ", ".join(products), xy=(v + shift, y1 + shift), fontsize=8, color='k'
        )

        ax.plot([x1, x2], [y1, y1], color='darkred', linewidth=2)
    else:
        ax.plot([x1, x2], [y1, y1], "k", linewidth = 2)


ax.set_xlabel("Potential vs Na/Na+, eV")
ax.set_ylabel("Na uptake per f.u.")

plt.tight_layout()


In [ ]:
limits

## Part 4. Screening of solid electrolytes for K-ion solid state batteries

A solid electrolyte should be:

- electronic insulator
- good ionic conductor

- stable vs. metal anode

Task:

- Collect mp_ids, chemical systems and band gap values for the stable K-ion containing structures with:
    - Eg > 2.0
    - Volume < 200 Å^3
    - < 4 elements
- Calculate electrochemical windows for the collected materials
- Collect data in the pandas DataFrame

- Identify the most promising candidates for the next stage of the screening

In [ ]:
with MPRester(api_key) as mpr:
    docs = mpr.materials.summary.search(
                              fields = [
                                        'material_id',
                                        'chemsys',
                                        'band_gap',
                                        ],
                              volume = (0, 200),
                              is_stable = True,
                              band_gap = (2.0, 1000.0),
                              possible_species = ['K+'],
                              exclude_elements = ['Pb'],
                              num_elements= (1, 4),
)

In [ ]:
keys = [
"material_id",
"band_gap",
"chemsys",
]

table = pd.DataFrame()

for key in keys:
    data = []
    for doc in docs:
        data.append(doc.__dict__[key])
    table[key] = data

In [ ]:
table

In [ ]:
from pymatgen.core import Element

def elchem_window(entry, entries, ref_element = 'K', tol = 1e-3):

    # construct phase diagram
    phase_diagram = PhaseDiagram([entry] + entries)
    ref_element_entries = [e for e in entries if e.composition.reduced_formula == ref_element]
    u0 = min(ref_element_entries, key=lambda e: e.energy_per_atom).energy_per_atom
    el_profile = phase_diagram.get_element_profile(Element(ref_element), entry.composition)
    limits = [None, None]
    for i, d in enumerate(el_profile):
        voltage = -(d["chempot"] - u0)
        evolution =  d["evolution"]
        if abs(evolution) < tol:
            print(d['reaction'], voltage)
            limits[0] = voltage
            limits[1] = float(-(el_profile[i+1]['chempot'] - u0))
    return limits

In [ ]:
mp_ids, reduction_limits, oxidation_limits = [], [], []

with MPRester(api_key = api_key) as mp_rester:
    for i, doc in enumerate(docs):
        mp_id = str(doc.material_id)
        chemsys = doc.chemsys.split('-')
        mp_ids.append(mp_id)
        entry = mp_rester.get_entry_by_material_id(mp_id)[0]
        entries = mp_rester.get_entries_in_chemsys(chemsys, compatible_only=True)
        limits = elchem_window(entry, entries, ref_element = 'K')
        reduction_limits.append(limits[0])
        oxidation_limits.append(limits[1])
        break

## Part 5. Other data 



### Density of states

In [ ]:
from mp_api.client import MPRester

with MPRester(api_key) as mpr:
    dos = mpr.get_dos_by_material_id("mp-149")

In [ ]:
dos.as_dict()['densities'].keys()

In [ ]:
fig, ax = plt.subplots(dpi = 150)

energies = dos.energies
density_up = dos.as_dict()['densities']['1']

efermi = dos.efermi
ax.plot(energies, density_up, label = 'Total DOS')
ax.set_xlabel('Energy, eV')
ax.set_ylabel('Density, states/eV')
ax.vlines(efermi, min(density_up), max(density_up), color = 'k', linestyle = '--', label = 'Fermi level')
ax.legend()


### Charge density

In [ ]:
from mp_api.client import MPRester

with MPRester(api_key) as mpr:
    chgcar = mpr.get_charge_density_from_material_id("mp-149")
data = (chgcar.data['total'] - chgcar.data['diff']) / 2

In [ ]:
coords = np.argwhere(chgcar.data['total'])

In [ ]:
x, y, z = coords[0, :], coords[1, :], coords[:, 2]
vol = data.reshape(-1,)

In [ ]:
import numpy as np
from numpy import cos, pi
from skimage.measure import marching_cubes
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

iso_val = 10
verts, faces, _, _ = marching_cubes(data, iso_val, spacing=(0.1, 0.1, 0.1))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(verts[:, 0], verts[:,1], faces, verts[:, 2], cmap='Spectral',
                lw=1)
plt.show()

### Phonons band structure

In [ ]:
from mp_api.client import MPRester

with MPRester(api_key) as mpr:
    ph_bs = mpr.get_phonon_bandstructure_by_material_id("mp-149")

In [ ]:
ph_bs.as_dict().keys()

In [ ]:
# ph_bs.eigendisplacements

In [ ]:

for q, band in zip(ph_bs.qpoints, ph_bs.bands):
    plt.plot(band)



### Phonon density of states

In [ ]:
from mp_api.client import MPRester
from emmet.core.electronic_structure import BSPathType

with MPRester(api_key) as mpr:
    ph_dos = mpr.get_phonon_dos_by_material_id("mp-149")

In [ ]:
plt.plot(ph_dos.frequencies, ph_dos.densities)